# Sequence to Sequence

### simple neural machine translation training

* sequence to sequence
  
### Reference
* [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215)

In [ ]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf

from matplotlib import font_manager, rc

# rc('font', family='AppleGothic') #for mac

import matplotlib.pyplot as plt



from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences

from pprint import pprint
import numpy as np
import os

print(tf.__version__)

In [ ]:
sources = [['I', 'feel', 'hungry'],
     ['tensorflow', 'is', 'very', 'difficult'],
     ['tensorflow', 'is', 'a', 'framework', 'for', 'deep', 'learning'],
     ['tensorflow', 'is', 'very', 'fast', 'changing']]
targets = [['나는', '배가', '고프다'],
           ['텐서플로우는', '매우', '어렵다'],
           ['텐서플로우는', '딥러닝을', '위한', '프레임워크이다'],
           ['텐서플로우는', '매우', '빠르게', '변화한다']]

In [ ]:
# vocabulary for sources
# 얻은 단어들을 이용하여 어휘집을 만들고, 이를 정렬한 뒤에 각 단어에 고유한 인덱스를 할당하는 과정을 수행
s_vocab = list(set(sum(sources, [])))
s_vocab.sort()
s_vocab = ['<pad>'] + s_vocab
source2idx = {word : idx for idx, word in enumerate(s_vocab)} # 각 단어에 대해 고유한 인덱스를 할당 -> 사전
idx2source = {idx : word for idx, word in enumerate(s_vocab)} # 인덱스를 키로 하고 단어를 값으로 하는 사전

pprint(source2idx)

In [ ]:
# vocabulary for targets
# 깃 데이터에서 사용되는 단어들로 어휘집을 만들고, 이에 인덱스를 할당

t_vocab = list(set(sum(targets, []))) # targets라는 리스트의 리스트에서 모든 단어들을 추출 & 중복 제거 -> 리스트화: t_vocab
t_vocab.sort() # 알파벳 순으로 정렬
t_vocab = ['<pad>', '<bos>', '<eos>'] + t_vocab # 특수 토큰 추가, '<pad>'는 패딩을 위한 토큰, '<bos>'와 '<eos>'는 각각 문장의 시작과 끝을 나타내는 토큰
target2idx = {word : idx for idx, word in enumerate(t_vocab)} # 각 단어에 대해 고유한 인덱스를 할당하는 사전
idx2target = {idx : word for idx, word in enumerate(t_vocab)} # 인덱스를 키로 하고 단어를 값으로 하는 사전

pprint(target2idx)

## sequence 데이터 전처리 (source, target 2가지의 작업 모드 지원)

### source 모드 (인코더를 위한 전처리)
1. 시퀀스 토큰화: 입력된 sequences의 각 문장을 토큰화하고, 이 토큰들을 dic 사전을 사용하여 해당 인덱스로 변환합니다.
2. 문장 길이 계산: 변환된 각 문장의 길이를 계산합니다.
3. 패딩 및 자르기: pad_sequences 함수를 사용하여 모든 문장을 max_len 길이로 맞춥니다. 길이가 max_len보다 짧은 문장은 뒤쪽에 패딩(<pad>)을 추가하고, 길이가 더 긴 문장은 뒷부분을 잘라냅니다.
4. 결과 반환: 각 문장의 길이와 전처리된 입력 데이터를 반환합니다.

### target 모드 (디코더를 위한 전처리)

1. 시퀀스 토큰화 및 특수 토큰 추가: 입력된 sequences의 각 문장 앞에 <bos>를 추가하고, 끝에 <eos>를 추가한 뒤 토큰화합니다. 그런 다음 dic 사전을 사용하여 이 토큰들을 해당 인덱스로 변환합니다.
2. 입력 문장 길이 계산: 변환된 각 입력 문장의 길이를 계산합니다.
3. 입력 데이터 패딩 및 자르기: 입력 데이터에 대해 pad_sequences 함수를 사용하여 max_len 길이로 맞춥니다.
4. 출력 데이터 준비: 원래 sequences의 각 문장 끝에 <eos>를 추가한 후 토큰화하고, dic 사전을 사용하여 인덱스로 변환합니다. 그 후 pad_sequences를 사용하여 max_len 길이로 맞춥니다.
5. 결과 반환: 각 입력 문장의 길이, 전처리된 입력 데이터, 전처리된 출력 데이터를 반환합니다.

In [ ]:
def preprocess(sequences, max_len, dic, mode = 'source'):
    assert mode in ['source', 'target'], 'source와 target 중에 선택해주세요.'

    if mode == 'source':
        # preprocessing for source (encoder)
        s_input = list(map(lambda sentence : [dic.get(token) for token in sentence], sequences))
        s_len = list(map(lambda sentence : len(sentence), s_input))
        s_input = pad_sequences(sequences = s_input, maxlen = max_len, padding = 'post', truncating = 'post')
        return s_len, s_input

    elif mode == 'target':
        # preprocessing for target (decoder)
        # input
        t_input = list(map(lambda sentence : ['<bos>'] + sentence + ['<eos>'], sequences))
        t_input = list(map(lambda sentence : [dic.get(token) for token in sentence], t_input))
        t_len = list(map(lambda sentence : len(sentence), t_input))
        t_input = pad_sequences(sequences = t_input, maxlen = max_len, padding = 'post', truncating = 'post')

        # output
        t_output = list(map(lambda sentence : sentence + ['<eos>'], sequences))
        t_output = list(map(lambda sentence : [dic.get(token) for token in sentence], t_output))
        t_output = pad_sequences(sequences = t_output, maxlen = max_len, padding = 'post', truncating = 'post')

        return t_len, t_input, t_output

In [ ]:
# preprocessing for source
s_max_len = 10
s_len, s_input = preprocess(sequences = sources,
                            max_len = s_max_len, dic = source2idx, mode = 'source')
print(s_len, s_input)

In [ ]:
# preprocessing for target
t_max_len = 12
t_len, t_input, t_output = preprocess(sequences = targets,
                                      max_len = t_max_len, dic = target2idx, mode = 'target')
print(t_len, t_input, t_output)

# hyper-param

In [ ]:
# hyper-parameters
epochs = 200
batch_size = 4
learning_rate = .005
total_step = epochs / batch_size
buffer_size = 100
n_batch = buffer_size//batch_size
embedding_dim = 32
units = 32

# input
data = tf.data.Dataset.from_tensor_slices((s_len, s_input, t_len, t_input, t_output)) #i/o 를 sequence 데이터 에서 받아옴.
data = data.shuffle(buffer_size = buffer_size)
data = data.batch(batch_size = batch_size)
# s_mb_len, s_mb_input, t_mb_len, t_mb_input, t_mb_output = iterator.get_next()

In [ ]:
def gru(units):
    return tf.keras.layers.GRU(units,
                               return_sequences=True,
                               return_state=True,
                               recurrent_initializer='glorot_uniform')

### Encoder - Seq2Seq Model
- vocab_size: 어휘집의 크기. 즉, 모델이 처리할 수 있는 고유 단어의 수.
- embedding_dim: 임베딩 차원. 각 단어를 나타내는 벡터의 크기.
- enc_units: 인코더의 유닛(또는 뉴런) 수. 이는 GRU 레이어의 복잡성을 결정합니다.
- batch_sz: 배치 크기. 모델이 한 번에 처리하는 데이터의 수.
이 함수는 임베딩 레이어와 GRU 레이어를 초기화합니다.

In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        # 입력된 단어 인덱스를 해당 임베딩 벡터로 변환 -> 단어의 의미를 수치적으로 표현하는 과정
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        # GRU (Gated Recurrent Unit) 레이어: 순차적인 데이터를 처리하기 위한 레이어 -> LSTM과 유사, 더 간단한 구조
        self.gru = gru(self.enc_units)

    # 임베딩 레이어를 통해 입력 데이터를 임베딩 벡터로 변환하고, GRU 레이어를 통해 출력(output)과 새로운 숨겨진 상태(state)를 생성
    # 모델이 입력 데이터 x와 숨겨진 상태 hidden을 받아 처리하는 함수
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)

        return output, state
    # 인코더의 초기 숨겨진 상태를 생성. 이는 모든 요소가 0인 텐서로, 배치 크기와 인코더 유닛 수에 따라 결정
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

### Decoder - Seq2Seq Model
- vocab_size: 어휘집의 크기. 즉, 모델이 처리할 수 있는 고유 단어의 수.
- embedding_dim: 임베딩 차원. 각 단어를 나타내는 벡터의 크기.
- enc_units: 인코더의 유닛(또는 뉴런) 수. 이는 GRU 레이어의 복잡성을 결정합니다.
- batch_sz: 배치 크기. 모델이 한 번에 처리하는 데이터의 수. 이 함수는 임베딩 레이어와 GRU 레이어를 초기화합니다.

In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        # 입력된 단어 인덱스를 해당 임베딩 벡터로 변환 -> 단어의 의미를 수치적으로 표현하는 과정
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        # GRU (Gated Recurrent Unit) 레이어: 순차적인 데이터를 처리하기 위한 레이어 -> LSTM과 유사, 더 간단한 구조
        self.gru = gru(self.dec_units)
        self.fc = tf.keras.layers.Dense(vocab_size)

    # 임베딩 레이어를 통해 입력 데이터를 임베딩 벡터로 변환하고, GRU 레이어를 통해 출력(output)과 새로운 숨겨진 상태(state)를 생성
    # 모델이 입력 데이터 x와 숨겨진 상태 hidden을 받아 처리하는 함수
    def call(self, x, hidden, enc_output):

        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)

        return x, state

    # 인코더의 초기 숨겨진 상태를 생성. 이는 모든 요소가 0인 텐서로, 배치 크기와 인코더 유닛 수에 따라 결정
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

In [ ]:
encoder = Encoder(len(source2idx), embedding_dim, units, batch_size)
decoder = Decoder(len(target2idx), embedding_dim, units, batch_size)

def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask

#     print("real: {}".format(real))
#     print("pred: {}".format(pred))
#     print("mask: {}".format(mask))
#     print("loss: {}".format(tf.reduce_mean(loss_)))

    return tf.reduce_mean(loss_)

# creating optimizer
optimizer = tf.keras.optimizers.Adam()

# creating check point (Object-based saving)
checkpoint_dir = './data_out/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                encoder=encoder,
                                decoder=decoder)

# create writer for tensorboard
#summary_writer = tf.summary.create_file_writer(logdir=checkpoint_dir)

### Seq2Seq
- 어려운이유, 모델 2개가 합쳐저셔 학습이 복잡한편
- incoder가 아무리 학습 & 압축을 잘한다고 해도 context에서 문제(데이터 손상)가 생기면 decoder 에서 (데이터가 잘 안받아지는)문제가 생길수 있음

In [ ]:
for epoch in range(epochs):
    # 매 에폭마다 인코더의 숨겨진 상태를 초기화합니다.
    hidden = encoder.initialize_hidden_state()
    # 에폭별 총 손실을 기록하기 위한 변수입니다.
    total_loss = 0

    # 데이터셋을 배치 단위로 순회하면서 각 배치에 대해 학습을 수행합니다.
    for i, (s_len, s_input, t_len, t_input, t_output) in enumerate(data):
        # 각 배치에 대한 손실을 초기화합니다.
        loss = 0
        with tf.GradientTape() as tape:
            # 인코더를 실행하여 출력과 숨겨진 상태를 얻습니다.
            enc_output, enc_hidden = encoder(s_input, hidden)

            # 디코더의 초기 숨겨진 상태를 설정합니다.
            dec_hidden = enc_hidden

            # 디코더의 초기 입력을 설정합니다. 이는 각 시퀀스의 시작을 나타내는 <bos> 토큰입니다.
            dec_input = tf.expand_dims([target2idx['<bos>']] * batch_size, 1)

            # Teacher Forcing 기법을 사용합니다. 이는 다음 입력으로 실제 타깃을 사용하는 방식입니다.
            for t in range(1, t_input.shape[1]):
                # 디코더를 실행하여 예측값과 새로운 숨겨진 상태를 얻습니다.
                predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_output)

                # 손실 함수를 사용하여 현재 시점의 손실을 계산합니다.
                loss += loss_function(t_input[:, t], predictions)

                # 다음 시점의 입력을 설정합니다. 이는 현재 타깃 시퀀스의 현재 토큰입니다.
                dec_input = tf.expand_dims(t_input[:, t], 1)

        # 배치에 대한 평균 손실을 계산합니다.
        batch_loss = (loss / int(t_input.shape[1]))

        # 에폭별 총 손실에 배치 손실을 추가합니다.
        total_loss += batch_loss

        # 인코더와 디코더의 변수를 결합합니다.
        variables = encoder.variables + decoder.variables

        # 손실에 대한 변수들의 그레이디언트를 계산합니다.
        gradient = tape.gradient(loss, variables)

        # 계산된 그레이디언트를 사용하여 변수들을 업데이트합니다.
        optimizer.apply_gradients(zip(gradient, variables))

    # 일정 에폭마다 모델의 진행 상황을 출력하고 모델을 저장합니다.
    if epoch % 10 == 0:
        print('Epoch {} Loss {:.4f} Batch Loss {:.4f}'.format(epoch,
                                            total_loss / n_batch,
                                            batch_loss.numpy()))
        checkpoint.save(file_prefix = checkpoint_prefix)


In [ ]:
#restore checkpoint

checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
sentence = 'I feel hungry'

In [ ]:
def prediction(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):

    inputs = [inp_lang[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang['<bos>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_out)

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += idx2target[predicted_id] + ' '

        if idx2target.get(predicted_id) == '<eos>':
            return result, sentence

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence

result, output_sentence = prediction(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)

print(sentence)
print(result)